In [1]:
import pandas as pd
import numpy as np

In [2]:
data1 = pd.read_excel("附件1.xlsx")
data1.head(1)

,单品编码,单品名称,分类编码,分类名称
0,102900005115168,牛首生菜,1011010101,花叶类


In [3]:
data2 = pd.read_excel("附件2.xlsx")
data2.head(2)

,销售日期,扫码销售时间,单品编码,销量(千克),销售单价(元/千克),销售类型,是否打折销售
0,2020-07-01,09:15:07.924,102900005117056,0.396,7.6,销售,否
1,2020-07-01,09:17:27.295,102900005115960,0.849,3.2,销售,否


In [4]:
date = data2["销售日期"]
date = date.drop_duplicates()

In [5]:
data3 = pd.read_excel("附件3.xlsx")
data3.head(1)

,日期,单品编码,批发价格(元/千克)
0,2020-07-01,102900005115762,3.88


In [6]:
code_list = []
date_list = list(pd.to_datetime(date).dt.date)

In [7]:
for i in data1["单品编码"]:
    code_list.append(i)

In [8]:
qua_code_list = [str(val) for val in code_list for i in range(4)]

In [9]:
data_new = {}
data_new = pd.DataFrame(data_new, index=pd.to_datetime(date).dt.date, columns=[qua_code_list, ['销量','批发价格','销售单价','是否打折'] * int(len(qua_code_list) / 4)])

In [10]:
print((pd.to_datetime(data2.iloc[878502,0])).date())
print(data_new.index[1084])
pd.to_datetime(data2.iloc[878502,0]).date() == data_new.index[1084]

2023-06-30
2023-06-30


True

In [11]:
data_new.columns[0]
data2.iloc[878502,0]

Timestamp('2023-06-30 00:00:00')

In [12]:
data_new.head(1)

102900005115168                102900005115199                 \
                        销量 批发价格 销售单价 是否打折              销量 批发价格 销售单价 是否打折   
销售日期                                                                       
2020-07-01             NaN  NaN  NaN  NaN             NaN  NaN  NaN  NaN   

           102900005115625       ... 106971533455008      106973223300667  \
                        销量 批发价格  ...            销售单价 是否打折              销量   
销售日期                             ...                                        
2020-07-01             NaN  NaN  ...             NaN  NaN             NaN   

                          106973990980123                 
           批发价格 销售单价 是否打折              销量 批发价格 销售单价 是否打折  
销售日期                                                      
2020-07-01  NaN  NaN  NaN             NaN  NaN  NaN  NaN  

[1 rows x 1004 columns]

In [13]:
np.isnan(data_new.iloc[1,1])

True

In [14]:
print("start to write sold data")
remove_list = code_list.copy()
for i in range(0, 878053):  # 读取附件2的每一行
    if i % 100000 == 0:
        print(i)
    read_time = pd.to_datetime(data2.iloc[i,0]).date()
    read_code = int(data2.iloc[i,2])
    read_data = data2.iloc[i,3]
    read_price = data2.iloc[i,4]
    read_discount = data2.iloc[i,6]

    j = date_list.index(read_time) # 寻找时间
    k = code_list.index(read_code) # 寻找单品
    sold = int(4 * k)
    price = sold + 2
    discount = price + 1
    exist_sold = data_new.iloc[j,sold]
    # 添加售出量
    if np.isnan(exist_sold):
        data_new.iloc[j,sold] = read_data
    else:
        data_new.iloc[j,sold] = read_data + exist_sold

    # 添加售价
    data_new.iloc[j,price] = read_price
    # 添加是否打折
    data_new.iloc[j,discount] = read_discount

    # 排除有售出记录项
    if data_new.iloc[j,sold] > 0:
        try:
            loca = remove_list.index(read_code)
            remove_list.pop(loca)
        except Exception:
            # a = 1
            None

    if date_list.index(pd.to_datetime(data2.iloc[i+1,0]).date()) == j+1 or j == 1084:
        for k in range(0,251):
            if np.isnan(data_new.iloc[j,sold]):
                data_new.iloc[j,sold] = 0

print(remove_list)
print("Done")

print("Start to write wholesale price")
lack_time_list = []
for i in range(0,55982):
    if i % 10000 == 0:
        print(i)
    read_time = pd.to_datetime(data3.iloc[i,0]).date()
    read_code = int(data3.iloc[i,1])
    read_buy = data3.iloc[i,2]

    try:
        j = date_list.index(read_time) # 寻找时间
    except Exception:
        try:
            x = lack_time_list.index(read_time)
            continue
        except Exception:
            lack_time_list.append(read_time)
            continue
    k = code_list.index(read_code) # 寻找单品

    buy = int(4 * k) + 1
    
    data_new.iloc[j,buy] = read_buy
print(lack_time_list)
print('Done')

start to write sold data
0
100000
200000
300000
400000
500000
600000
700000
800000
[102900005116776, 102900005116042, 102900011023648, 102900011032145, 102900011011782]
Done
Start to write wholesale price
0
10000
20000
30000
40000
50000
[datetime.date(2022, 11, 2), datetime.date(2022, 11, 4), datetime.date(2022, 11, 30), datetime.date(2022, 12, 1), datetime.date(2022, 12, 2), datetime.date(2022, 12, 3)]
Done


In [15]:
data_new.to_excel('每日各种单品数据.xlsx')

In [16]:
data_new

102900005115168                102900005115199                 \
                        销量 批发价格 销售单价 是否打折              销量 批发价格 销售单价 是否打折   
销售日期                                                                       
2020-07-01             NaN  NaN  NaN  NaN             NaN  NaN  NaN  NaN   
2020-07-02             NaN  NaN  NaN  NaN             NaN  NaN  NaN  NaN   
2020-07-03             NaN  NaN  NaN  NaN             NaN  NaN  NaN  NaN   
2020-07-04             NaN  NaN  NaN  NaN             NaN  NaN  NaN  NaN   
2020-07-05             NaN  NaN  NaN  NaN             NaN  NaN  NaN  NaN   
...                    ...  ...  ...  ...             ...  ...  ...  ...   
2023-06-26             NaN  NaN  NaN  NaN             NaN  NaN  NaN  NaN   
2023-06-27             NaN  NaN  NaN  NaN             NaN  NaN  NaN  NaN   
2023-06-28             NaN  NaN  NaN  NaN             NaN  NaN  NaN  NaN   
2023-06-29             NaN  NaN  NaN  NaN             NaN  NaN  NaN  NaN   
2023-06-30             NaN  NaN  NaN  NaN             NaN  NaN  NaN  NaN   

           102900005115625       ... 106971533455008      106973223300667  \
                        销量 批发价格  ...            销售单价 是否打折              销量   
销售日期                             ...                                        
2020-07-01             NaN  NaN  ...             NaN  NaN             NaN   
2020-07-02             NaN  NaN  ...             NaN  NaN             NaN   
2020-07-03             NaN  NaN  ...             NaN  NaN             NaN   
2020-07-04             NaN  NaN  ...             NaN  NaN             NaN   
2020-07-05             NaN  NaN  ...             NaN  NaN             NaN   
...                    ...  ...  ...             ...  ...             ...   
2023-06-26             NaN  NaN  ...             NaN  NaN             NaN   
2023-06-27             NaN  NaN  ...             NaN  NaN             NaN   
2023-06-28             NaN  NaN  ...             NaN  NaN             NaN   
2023-06-29             NaN  NaN  ...             NaN  NaN             NaN   
2023-06-30             NaN  NaN  ...             NaN  NaN             NaN   

                          106973990980123                 
           批发价格 销售单价 是否打折              销量 批发价格 销售单价 是否打折  
销售日期                                                      
2020-07-01  NaN  NaN  NaN             NaN  NaN  NaN  NaN  
2020-07-02  NaN  NaN  NaN             NaN  NaN  NaN  NaN  
2020-07-03  NaN  NaN  NaN             NaN  NaN  NaN  NaN  
2020-07-04  NaN  NaN  NaN             NaN  NaN  NaN  NaN  
2020-07-05  NaN  NaN  NaN             NaN  NaN  NaN  NaN  
...         ...  ...  ...             ...  ...  ...  ...  
2023-06-26  NaN  NaN  NaN             NaN  NaN  NaN  NaN  
2023-06-27  NaN  NaN  NaN             NaN  NaN  NaN  NaN  
2023-06-28  NaN  NaN  NaN             NaN  NaN  NaN  NaN  
2023-06-29  NaN  NaN  NaN             NaN  NaN  NaN  NaN  
2023-06-30  NaN  NaN  NaN             NaN  NaN  NaN  NaN  

[1085 rows x 1004 columns]